In [1]:
#!/usr/bin/evn python

"""
CMSC733 Spring 2019: Classical and Deep Learning Approaches for
Geometric Computer Vision
Project1: MyAutoPano: Phase 1 Starter Code

Author(s): 
Chahat Deep Singh (chahat@terpmail.umd.edu) 
PhD Student in Computer Science,
University of Maryland, College Park

Nitin J. Sanket (nitinsan@terpmail.umd.edu)
PhD Candidate in Computer Science,
University of Maryland, College Park
"""

#Code starts here:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage.feature import peak_local_max

In [2]:
def readImageSet(folder_name):
    print("Reading images from ", folder_name)
    images = []
    files = os.listdir(folder_name)	
    files = sorted(files)
    print("Found ", files)
    for file in files:
        image_path = folder_name + "/" + file
        image = cv2.imread(image_path)
        if image is not None:
            images.append(image)			
        else:
            print("Error in loading image ", image)

    return images


In [3]:
def displayImages(image_array, name):

    image_array = makeImageSizeSame(image_array)
    concat = image_array[0].copy()

    for l in range(1,len(image_array)):
        image = image_array[l]
        concat = np.concatenate((concat,image), axis = 1)
        
    cv2.imshow(name, concat)
    cv2.waitKey() 
    cv2.destroyAllWindows()


In [4]:
def detectCorners(images, choice):
    print("detecting corners ...")
    detected_corners = []
    cmaps = []
    corner_images = []
    for i in images:
        image = i.copy()
        gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        gray_image = np.float32(gray_image)


        if(choice == 1):
            print("using Harris corner detection method.")
            corner_strength = cv2.cornerHarris(gray_image,2,3,0.04)
            corner_strength[corner_strength<0.01*corner_strength.max()] = 0
            detected_corner = np.where(corner_strength>0.001*corner_strength.max())
            detected_corners.append(detected_corner)
            cmaps.append(corner_strength)
            image[corner_strength > 0.001*corner_strength.max()]=[0,0,255]
            corner_images.append(image)
        else:
            print("using Shi-Tomashi corner detection method.")
            dst = cv2.goodFeaturesToTrack(gray_image, 1000 ,0.01, 10)
            dst = np.int0(dst)
            detected_corners.append(dst)
            for c in dst:
                x,y = c.ravel()
                cv2.circle(image,(x,y),3,(0, 0, 255),-1) 
                          
            corner_images.append(image)
            cmap = np.zeros(gray_image.shape) #not sure what to do
            cmaps.append(cmap)
    #filter detected corners
    #remove the corner one
    return detected_corners, cmaps, corner_images

In [5]:
def getFeatureDescriptor(gray_image,x,y, patch_size=40):
    patch = gray_image[x-patch_size//2:x+patch_size//2, y-patch_size//2:y+patch_size//2] 
    # gaussian blur
    patch = cv2.GaussianBlur(patch,(3,3),0)
    # subsample to 20% size or 1/5th
    patch = cv2.resize(patch, None, fx=0.2, fy=0.2, interpolation = cv2.INTER_CUBIC)
    feature = patch.reshape(-1)
    feature = (feature-feature.mean())/ np.std(feature)
    return feature

In [6]:
def getPairs(image_1, image_2, all_corners_1, all_corners_2, patch_size = 40, alpha = 0.8):

    gray_image1 = cv2.cvtColor(image_1,cv2.COLOR_BGR2GRAY)
    gray_image2 = cv2.cvtColor(image_2,cv2.COLOR_BGR2GRAY)

    width1, height1 = image_1.shape[:2]
    width2, height2 = image_2.shape[:2]

    print("width = ", width1, ", height = ", height1)
    print("width = ", width2, ", height = ", height2)

    features_1,features_2 = [], []
    corners_1, corners_2 = [],[]

    print("all corner 1 len = ", len(all_corners_1))
    print("all corner 2 len = ", len(all_corners_2))
    
    for corner in all_corners_1:
        x,y = corner.ravel()
        
        if (x - (patch_size / 2) > 0) & (x + (patch_size / 2) < height1) & (y - (patch_size / 2) > 0) & (y + (patch_size / 2) < width1):
            features_1.append(getFeatureDescriptor(gray_image1, y,x)) 
            corners_1.append([x,y])
        else:
            #print("ignored x, y", x, y)
            pass

    for corner in all_corners_2:
        x,y = corner.ravel()
        if (x - (patch_size / 2) > 0) & (x + (patch_size / 2) < height2) & (y - (patch_size / 2) > 0) & (y + (patch_size / 2) < width2):
            features_2.append(getFeatureDescriptor(gray_image2, y,x)) 
            corners_2.append([x,y]) 

    matched_pairs, match_level = [], []
    for i, feat_1 in enumerate(features_1):
        ssd = []  
        for j, feat_2 in enumerate(features_2):
            ssd.append(np.sum((feat_1 - feat_2)**2))
        top_matche = np.argmin(ssd)
        #if ssd[top_matches[0]] / ssd[top_matches[1]] < alpha:   
            #matched_pairs.append([corners_1[i] , corners_2[top_matches[0]]])
        matched_pairs.append([corners_1[i] , corners_2[top_matche]]) 
    print("matched pairs num = ", len(matched_pairs))
    matched_pairs = np.array(matched_pairs)
    return matched_pairs

In [7]:
def makeImageSizeSame(images):
    sizes = []
    for image in images:
        x, y, ch = image.shape
        sizes.append([x, y, ch])

    sizes = np.array(sizes)
    x_target, y_target, _ = np.max(sizes, axis = 0)
    
    images_resized = []

    for i, image in enumerate(images):
        image_resized = np.zeros((x_target, y_target, sizes[i, 2]), np.uint8)
        image_resized[0:sizes[i, 0], 0:sizes[i, 1], 0:sizes[i, 2]] = image
        images_resized.append(image_resized)

    return images_resized

In [8]:
def showMatches(image_1, image_2, matched_pairs, partition_width = 20):

    image_1, image_2 = makeImageSizeSame([image_1, image_2])

    concat = np.concatenate((image_1, image_2), axis = 1)
    corners_1 = matched_pairs[:,0].copy()
    corners_2  = matched_pairs[:,1].copy()
    corners_2[:,0] += image_1.shape[1]

    for (x1,y1) , (x2,y2) in zip(corners_1, corners_2):
        cv2.line(concat, (x1,y1), (x2,y2), (0, 0, 255), 1)
    
      
    cv2.imshow('Feature_matches', concat)
    cv2.waitKey() 
    cv2.destroyAllWindows()


In [9]:
def testShowMatches(image_1, image_2, partition_width = 20):
    matched_pairs= []
    I = np.linspace(10, 100, 10)
    for i in I:
        x1 = i
        y1 = i
        corner1 = np.int0(np.array([x1, y1]))

        x2 = i
        y2 = i
        corner2 = np.int0(np.array([x2, y2]))
        matched_pairs.append([corner1 , corner2])

    matched_pairs = np.array(matched_pairs)
    showMatches(image_1, image_2, matched_pairs, partition_width = 20)


In [10]:
def filterOutliers(matched_pairs, outliers, accuracy, thresh):

    set1 = matched_pairs[:, 0]
    set2 = matched_pairs[:, 1]

    N_best = 0
    H_best = np.zeros([3, 3])
    
    e = outliers / set1.shape[0]
    s = 4
    p = accuracy
    iterations = np.log(1 - p) / np.log(1 - np.power((1 - e), s))
    iterations = np.int(iterations)
    iterations = 4000

    filtered_pair_indices = []

    print("iterations = ", iterations)
    for i in range(iterations):
        #randomly select four points
        n_rows = set1.shape[0]
        random_indices = np.random.choice(n_rows, size=4)

        set1_random = set1[random_indices]
        set2_random = set2[random_indices]
              
        #compute homography
        H = cv2.getPerspectiveTransform(np.float32(set1_random), np.float32(set2_random))

        set1_dash = np.vstack((set1[:,0], set1[:,1], np.ones([1, n_rows])))
        set1_transformed_dash = np.dot(H, set1_dash)
        
        t1 = set1_transformed_dash[0,:]/set1_transformed_dash[2,:]
        t2 = set1_transformed_dash[1,:]/set1_transformed_dash[2,:]

        set1_transformed = np.array([t1, t2]).T
        #print(set1_transformed.shape)

        E = calculateError(set2, set1_transformed)
        
     
        E[E <= thresh] = 1
        E[E > thresh] = 0
    
    
        N = np.sum(E)


        if N > N_best:
            N_best = N
            H_best = H
            filtered_pair_indices = np.where(E == 1)
    
    filtered_set1 =  set1[filtered_pair_indices]
    filtered_set2 =  set2[filtered_pair_indices]

    print("Number of pairs after filtering = ", filtered_set1.shape[0])

    filter_matched_pairs = np.zeros([filtered_set1.shape[0], filtered_set1.shape[1], 2])

    filter_matched_pairs[:, 0, :] = filtered_set1
    filter_matched_pairs[:, 1, :] = filtered_set2

    filter_matched_pairs = filter_matched_pairs.astype(int)

    H_inbuit, _ = cv2.findHomography(set1, set2)
    print("inbuilt = ", H_inbuit)
    print("Computed = ", H_best)

    return H_best, filter_matched_pairs

    

In [11]:
def calculateError(set1, set2):
   
    E = np.zeros(set1.shape[0])
    tmp = set2 - set1
    num = set1.shape[0]

    for n in range(num):
        E[n] = np.linalg.norm(tmp[n])
    return E


In [12]:
def AdaptiveNonMaximalSuppression(images, C_maps, N_best):

    anms_img = []
    anms_corners = []
    for i,image in enumerate(images):

        cmap = C_maps[i]
        local_maximas = peak_local_max(cmap, min_distance=10)
        n_strong = local_maximas.shape[0]
        
        r = [np.Infinity for i in range(n_strong)]
        x=np.zeros((n_strong,1))
        y=np.zeros((n_strong,1))
        eu_dist = 0

        for i in range(n_strong):
            for j in range(n_strong):
                x_j = local_maximas[j][0]
                y_j = local_maximas[j][1]

                x_i = local_maximas[i][0]
                y_i = local_maximas[i][1]

                if(cmap[x_j, y_j] > cmap[x_i, y_i]):
                    eu_dist = np.square(x_j - x_i) + np.square(y_j - y_i)
                if r[i] > eu_dist:
                    r[i] = eu_dist
                    x[i] = x_j
                    y[i] = y_j

        index = np.argsort(r)
        index = np.flip(index)
        index = index[0:N_best]
        x_best=np.zeros((N_best,1))
        y_best=np.zeros((N_best,1))

        for i in range(N_best):
            x_best[i] = np.int0(x[index[i]])
            y_best[i] = np.int0(y[index[i]]) 
            cv2.circle(image, (y_best[i], x_best[i]), 3, (0, 255, 0), -1)

        anms_corner = np.int0(np.concatenate((x_best, y_best), axis = 1))
        anms_corners.append(anms_corner)
        anms_img.append(image)
    return anms_corners, anms_img

In [13]:
def stitchImagePairs(img0, img1, H):

    image0 = img0.copy()
    image1 = img1.copy()

    #stitch image 0 on image 1
    print("shapes")
    print(image0.shape)
    print(image1.shape)
    

    h0 ,w0 ,_ = image0.shape
    h1 ,w1 ,_ = image1.shape

    points_on_image0 = np.float32([[0, 0], [0, h0], [w0, h0], [w0, 0]]).reshape(-1,1,2)
    points_on_image0_transformed = cv2.perspectiveTransform(points_on_image0, H)
    print("transformed points = ", points_on_image0_transformed)
    points_on_image1 = np.float32([[0, 0], [0, h1], [w1, h1], [w1, 0]]).reshape(-1,1,2)

    points_on_merged_images = np.concatenate((points_on_image0_transformed, points_on_image1), axis = 0)
    points_on_merged_images_ = []

    for p in range(len(points_on_merged_images)):
        points_on_merged_images_.append(points_on_merged_images[p].ravel())

    points_on_merged_images_ = np.array(points_on_merged_images_)

    x_min, y_min = np.int0(np.min(points_on_merged_images_, axis = 0))
    x_max, y_max = np.int0(np.max(points_on_merged_images_, axis = 0))

    print("min, max")
    print(x_min, y_min)
    print(x_max, y_max)

    # overlap_area = cv2.polylines(image1,[np.int32(points_on_image0_transformed)],True,255,3, cv2.LINE_AA) 
    # cv2.imshow("original_image_overlapping.jpg", overlap_area)
    # cv2.waitKey() 
    # cv2.destroyAllWindows()
    H_translate = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]]) # translate

    image0_transformed_and_stitched = cv2.warpPerspective(image0, np.dot(H_translate, H), (x_max-x_min, y_max-y_min))
    image0_transformed_and_stitched[-y_min:-y_min+h1, -x_min: -x_min+w1] = image1
    return image0_transformed_and_stitched

In [14]:
def TransformImage(image, H): 

    image0 = image.copy()
    h0 ,w0 ,_ = image0.shape

    points_on_image0 = np.float32([[0, 0], [0, h0], [w0, h0], [w0, 0]]).reshape(-1,1,2)
    points_on_image0_transformed = cv2.perspectiveTransform(points_on_image0, H)

    for p in range(len(points_on_image0_transformed)):
        points_on_image0_transformed_.append(points_on_image0_transformed[p].ravel())

    points_on_image0_transformed_ = np.array(points_on_image0_transformed_)

    x_min, y_min = np.int0(np.min(points_on_image0_transformed, axis = 0))
    x_max, y_max = np.int0(np.max(points_on_image0_transformed, axis = 0))
    
    H_translate = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]]) # translate

    image0_transformed = cv2.warpPerspective(image0, np.dot(H_translate, H), (x_max-x_min, y_max-y_min))
    image0_transformed[-y_min:-y_min+h1, -x_min: -x_min+w1] = image1
    return image0_transformed

In [15]:
def stitchImageArray(image_array, H_array):
    
    N = len(image_array)

    target_image = image_array[N-1]

    ht, wt, _ = target_image.shape
    points_on_target_image = np.float32([[0, 0], [0, ht], [wt, ht], [wt, 0]]).reshape(-1,1,2)

    points = points_on_target_image
    
    images_transformed = []
    H_translate_array = []

    for n in range(N-1):
        image = image_array[n]
        h, w, _ = image.shape
        points_on_image = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1,1,2)

        points_on_image_transformed = points_on_image 
        for m in range(n, N-1):
            H = H_array[m]
            points_on_image_transformed = cv2.perspectiveTransform(points_on_image_transformed, H)
        
        points = np.concatenate((points, points_on_image_transformed), axis = 0)

    points_ = []
    for p in range(len(points)):
        points_.append(points[p].ravel())

    points_ = np.array(points_)

    x_min, y_min = np.int0(np.min(points_, axis = 0))
    x_max, y_max = np.int0(np.max(points_, axis = 0))

    H_translate = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]]) # translate

    for n in range(N-1):
        image = image_array[n]
        image_transformed = image
        for m in range(n, N-1):
            H = H_array[m]
            image_transformed = cv2.warpPerspective(image_transformed, np.dot(H_translate,H), (x_max-x_min, y_max-y_min))

        #image_transformed = cv2.warpAffine(image_transformed, H_translate, (x_max-x_min, y_max-y_min))
        images_transformed.append(image_transformed)

    final_output = np.zeros((y_max-y_min, x_max-x_min, 3), np.uint8)
    for i in range(len(images_transformed)):
        final_output = final_output + images_transformed[i]

    return images_transformed

In [16]:
def cropImage(img):

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    _,thresh = cv2.threshold(gray,5,255,cv2.THRESH_BINARY)
    kernel = np.ones((5,5), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contours[0])
        crop = img[y:y+h,x:x+w]
    return crop


In [17]:
def joinImages(img_array):
    image_array = img_array.copy()
    N = len(image_array)
    image0 = image_array[0]
    for i in range(1, N):

        print("processing image ", i)
        image1 = image_array[i] 

        image_pair = [image0, image1]
        
        detected_corners, cmaps, corner_images = detectCorners(image_pair, choice)
        displayImages(corner_images, "corners")
        """
        Perform ANMS: Adaptive Non-Maximal Suppression
        Save ANMS output as anms.png
        """
        if (choice == 1):
            print("Applying ALMS.")
            detected_corners, anms_image = AdaptiveNonMaximalSuppression(image_pair, cmaps, 500)
            displayImages(anms_image, "anms_output")
        else:
            print("goodFeaturesToTrack is already using ALMS.") #review

        detected_corners0 = detected_corners[0]
        detected_corners1 = detected_corners[1]
                
        matched_pairs = getPairs(image0, image1, detected_corners0, detected_corners1, patch_size = 40, alpha = 0.9 )
        showMatches(image0, image1, matched_pairs, partition_width = 20)
        """
        Refine: RANSAC, Estimate Homography
        """
        H,filtered_matched_pairs = filterOutliers(matched_pairs, 20, 0.9, 15)
        showMatches(image0, image1, filtered_matched_pairs, partition_width = 20)
        """
        Image Warping + Blending
        Save Panorama output as mypano.png
        """
        stitched_image = stitchImagePairs(image0, image1, H)
        stitched_image = cropImage(stitched_image)
        cv2.imshow("pano.jpg", stitched_image)
        cv2.waitKey() 
        cv2.destroyAllWindows()
        cv2.imwrite("Results/pano.png", stitched_image)
        image0 = stitched_image

    return image0


In [46]:
def main():
    # Add any Command Line arguments here
    # Parser = argparse.ArgumentParser()
    # Parser.add_argument('--NumFeatures', default=100, help='Number of best features to extract from each image, Default:100')
    
    # Args = Parser.parse_args()
    # NumFeatures = Args.NumFeatures
    

    """
    Read a set of images for Panorama stitching
    """
    print("main")
    folder_name = "/home/sakshi/courses/CMSC733/sakshi_p1/Phase1/Data/Train/out"
    use_harris = False
    images = readImageSet(folder_name)
    displayImages(images, "image_set")
    N_images = len(images)

    choice = 2
    if use_harris:
        choice = 1

    image0 = images[0]

    for i in range(1, N_images):

        print("processing image ", i)
        image1 = images[i] 

        image_pair = [image0, image1]
        
        detected_corners, cmaps, corner_images = detectCorners(image_pair, choice)
        #displayImages(corner_images, "corners")
        """
        Perform ANMS: Adaptive Non-Maximal Suppression
        Save ANMS output as anms.png
        """
        if (choice == 1):
            print("Applying ALMS.")
            detected_corners, anms_image = AdaptiveNonMaximalSuppression(image_pair, cmaps, 500)
            #displayImages(anms_image, "anms_output")
        else:
            print("goodFeaturesToTrack is already using ALMS.") #review

        detected_corners0 = detected_corners[0]
        detected_corners1 = detected_corners[1]
            
        matched_pairs = getPairs(image0, image1, detected_corners0, detected_corners1, patch_size = 40, alpha = 0.9 )
        #showMatches(image0, image1, matched_pairs, partition_width = 20)
        """
        Refine: RANSAC, Estimate Homography
        """
        H,filtered_matched_pairs = filterOutliers(matched_pairs, 20, 0.9, 25)
        #showMatches(image0, image1, filtered_matched_pairs, partition_width = 20)
        """
        Image Warping + Blending
        Save Panorama output as mypano.png
        """
        stitched_image = stitchImagePairs(image0, image1, H)
        stitched_image = cropImage(stitched_image)
        # cv2.imshow("pano.jpg", stitched_image)
        # cv2.waitKey() 
        # cv2.destroyAllWindows()
        cv2.imwrite("Results/pano.png", stitched_image)
        image0 = stitched_image
        


In [19]:
if __name__ == '__main__':
    main()

main
Reading images from  /home/sakshi/courses/CMSC733/sakshi_p1/Phase1/Data/Train/out


FileNotFoundError: [Errno 2] No such file or directory: '/home/sakshi/courses/CMSC733/sakshi_p1/Phase1/Data/Train/out'

In [57]:
print("main")
folder_name = "/home/sakshi/courses/CMSC733/sakshi_p1/Phase1/Data/Train/Set1"
use_harris = False
images = readImageSet(folder_name)
displayImages(images, "image_set")
N_images = len(images)
choice = 2
if use_harris:
    choice = 1

#image0 = images[0]

main
Reading images from  /home/sakshi/courses/CMSC733/sakshi_p1/Phase1/Data/Train/Set1
Found  ['1.jpg', '2.jpg', '3.jpg']


In [58]:
N_first_half = round(N_images/2)
N_second_half = N_images - N_first_half
print(N_first_half, " and ", N_second_half)

N_first_half_images = []
N_second_half_images = []

2  and  1


In [59]:

while N_images is not 2:
    print("N = ", N_images, " N_half = ", N_first_half)
    merged_images = []
    for n in range(0, N_first_half, 2):
        if (n+1) <= N_first_half:
            img_array = images[n:n+2]
            print("combining: ", n, n+1)
            I = joinImages(img_array)
            merged_images.append(I)
        else:
            print("adding: ", n)
            merged_images.append(images[n])

    for n in range(N_second_half, N_images, 2):
        if (n+1) <= N_images:
            img_array = images[n:n+2]
            img_array.reverse()
            print("combining: ", n+1, n)
            I = joinImages(img_array)
            merged_images.append(I)
        else:
            print("adding: ", n, n)
            merged_images.append(images[n])
 
    images = merged_images
    N_images = len(images)
    N_first_half = round(N_images/2)
    N_second_half = N_images - N_first_half


final = joinImages(merged_images)

          

N =  3  N_half =  2
combining:  0 1
processing image  1
detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection method.
goodFeaturesToTrack is already using ALMS.
width =  450 , height =  600
width =  450 , height =  600
all corner 1 len =  616
all corner 2 len =  711
matched pairs num =  517
iterations =  4000
Number of pairs after filtering =  229
inbuilt =  [[-6.51661112e-01 -2.28634018e+00  6.49829969e+02]
 [-1.86012425e-01 -6.53463614e-01  1.85314843e+02]
 [-9.86728387e-04 -3.54897705e-03  1.00000000e+00]]
Computed =  [[ 1.08974381e+00  8.08603106e-02 -3.67773262e+01]
 [-7.92567896e-03  1.10399181e+00 -2.50788461e+02]
 [-4.07087833e-05  2.98362149e-04  1.00000000e+00]]
shapes
(450, 600, 3)
(450, 600, 3)
transformed points =  [[[-3.6777325e+01 -2.5078847e+02]]

 [[-3.4399995e-01  2.1688785e+02]]

 [[ 5.8878528e+02  2.1737633e+02]]

 [[ 6.3251837e+02 -2.6194186e+02]]]
min, max
-36 -261
632 450
combining:  2 1
processing image  1
detecting 

In [ ]:
for n in range(0, N_first_half, 2):
    if (n+1) <= N_first_half:
        img_array = images[n:n+2]
        I = joinImages(img_array)
        N_first_half_images.append(I)
    else:
        N_first_half_images.append(images[n])

In [38]:
for n in range(N_second_half, N_images, 2):
    if (n+1) <= N_images:
        img_array = images[n:n+2]
        img_array.reverse()
        I = joinImages(img_array)
        N_second_half_images.append(I)
    else:
        N_second_half_images.append(images[n])

processing image  1
detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection method.
goodFeaturesToTrack is already using ALMS.
width =  807 , height =  605
width =  807 , height =  605
all corner 1 len =  360
all corner 2 len =  435
matched pairs num =  335
iterations =  4000
Number of pairs after filtering =  132
inbuilt =  [[-1.06160707e+03  8.11626687e+02  6.08756345e+03]
 [-1.60875462e+03  1.23145835e+03  8.14117585e+03]
 [-3.52523345e+00  2.71748615e+00  1.00000000e+00]]
Computed =  [[ 7.34088829e-01 -1.19466047e-02  2.10696588e+02]
 [-1.46420120e-01  9.07684534e-01  3.81698310e+01]
 [-3.89212476e-04 -7.70660364e-06  1.00000000e+00]]
shapes
(807, 605, 3)
(807, 605, 3)
transformed points =  [[[210.6966   38.16983]]

 [[202.31392 775.4942 ]]

 [[850.81537 899.48645]]

 [[856.50446 -65.94192]]]
min, max
0 -65
856 899
processing image  1
detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection metho

In [39]:
len(N_first_half_images)

0

In [25]:
img_array1 = images[0:2]
I1 = joinImages(img_array1)

processing image  1
detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection method.
goodFeaturesToTrack is already using ALMS.
width =  807 , height =  605
width =  807 , height =  605
all corner 1 len =  489
all corner 2 len =  561
matched pairs num =  442
iterations =  4000
Number of pairs after filtering =  117
inbuilt =  [[ 8.98932359e-01 -8.09269316e-01  2.35080040e+02]
 [ 1.94672202e+00 -1.72142241e+00  4.96172856e+02]
 [ 3.89482378e-03 -3.46407303e-03  1.00000000e+00]]
Computed =  [[ 1.24737379e+00 -8.84943188e-02 -3.29908594e+02]
 [ 1.02994597e-01  1.15563818e+00 -4.28231262e+01]
 [ 4.47103777e-04 -7.36595427e-05  1.00000000e+00]]
shapes
(807, 605, 3)
(807, 605, 3)
transformed points =  [[[-329.9086    -42.823128]]

 [[-426.6872    946.01086 ]]

 [[ 291.76028   786.1649  ]]

 [[ 334.3198     15.339346]]]
min, max
-426 -42
605 946


In [26]:
img_array2 = images[2:4]
I2 = joinImages(img_array2)


processing image  1
detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection method.
goodFeaturesToTrack is already using ALMS.
width =  807 , height =  605
width =  807 , height =  605
all corner 1 len =  429
all corner 2 len =  422
matched pairs num =  386
iterations =  4000
Number of pairs after filtering =  186
inbuilt =  [[-7.28248872e+00 -3.45472708e-01  8.90699601e+02]
 [-1.76804179e+01 -1.22585004e+00  2.44685023e+03]
 [-9.89539929e-03 -1.22353055e-04  1.00000000e+00]]
Computed =  [[ 1.28889178e+00  2.46378505e-02 -2.56829274e+02]
 [ 1.53408301e-01  1.17176000e+00 -5.61976774e+01]
 [ 5.42031499e-04 -8.65526178e-05  1.00000000e+00]]
shapes
(807, 605, 3)
(807, 605, 3)
transformed points =  [[[-256.82928   -56.197678]]

 [[-254.73956   956.20135 ]]

 [[ 431.47696   780.7324  ]]

 [[ 393.80887    27.572515]]]
min, max
-256 -56
605 956


In [27]:
img_array3 = images[4:6]
img_array3.reverse()
I3 = joinImages(img_array3)

processing image  1
detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection method.
goodFeaturesToTrack is already using ALMS.
width =  807 , height =  605
width =  807 , height =  605
all corner 1 len =  360
all corner 2 len =  435
matched pairs num =  335
iterations =  4000
Number of pairs after filtering =  130
inbuilt =  [[-1.06160707e+03  8.11626687e+02  6.08756345e+03]
 [-1.60875462e+03  1.23145835e+03  8.14117585e+03]
 [-3.52523345e+00  2.71748615e+00  1.00000000e+00]]
Computed =  [[ 7.31079802e-01 -4.76643136e-03  1.95554946e+02]
 [-1.53461429e-01  8.42666605e-01  5.40036140e+01]
 [-4.01958992e-04 -6.82421939e-05  1.00000000e+00]]
shapes
(807, 605, 3)
(807, 605, 3)
transformed points =  [[[195.55495   54.003613]]

 [[202.88141  776.8159  ]]

 [[903.4809   913.7121  ]]

 [[842.81946  -51.321075]]]
min, max
0 -51
903 913


In [28]:
img_array4 = images[6:8]
img_array4.reverse()
I4 = joinImages(img_array4)

processing image  1
detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection method.
goodFeaturesToTrack is already using ALMS.
width =  807 , height =  605
width =  807 , height =  605
all corner 1 len =  338
all corner 2 len =  371
matched pairs num =  303
iterations =  4000
Number of pairs after filtering =  213
inbuilt =  [[-5.64704695e-01 -1.41582635e-01  3.61545004e+02]
 [-9.46315307e-01 -1.57742839e-01  5.46116206e+02]
 [-1.62438234e-03 -3.54812627e-04  1.00000000e+00]]
Computed =  [[ 9.09417792e-01  3.37706749e-02  1.28406299e+02]
 [-6.45282732e-02  9.83695166e-01  1.05864661e+01]
 [-2.31417699e-04  4.67652520e-05  1.00000000e+00]]
shapes
(807, 605, 3)
(807, 605, 3)
transformed points =  [[[128.4063    10.586466]]

 [[149.99837  775.17377 ]]

 [[786.2671   852.5807  ]]

 [[789.08154  -33.085342]]]
min, max
0 -33
789 852


In [29]:
IC12 = [I1, I2]
IC1 = joinImages(IC12)

processing image  1
detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection method.
goodFeaturesToTrack is already using ALMS.
width =  988 , height =  1031
width =  1012 , height =  861
all corner 1 len =  790
all corner 2 len =  595
matched pairs num =  773
iterations =  4000
Number of pairs after filtering =  22
inbuilt =  [[ 5.88467183e-02 -7.45742259e-01  4.35425770e+02]
 [ 7.29205167e-02 -9.25691400e-01  5.40319259e+02]
 [ 1.35208058e-04 -1.71334941e-03  1.00000000e+00]]
Computed =  [[ 1.33506902e+00 -2.50732421e-01 -7.90384054e+02]
 [ 3.62198976e-01  5.13479223e-01 -6.00996968e+01]
 [ 6.13564893e-04 -1.13412748e-03  1.00000000e+00]]
shapes
(988, 1031, 3)
(1012, 861, 3)
transformed points =  [[[ -790.38403    -60.099697]]

 [[ 8613.718    -3710.7979  ]]

 [[  660.7498    1602.611   ]]

 [[  358.98404    191.92102 ]]]
min, max
-790 -3710
8613 1602


In [31]:
IC43 = [I4, I3]
IC2 = joinImages(IC43)

processing image  1
detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection method.
goodFeaturesToTrack is already using ALMS.
width =  885 , height =  789
width =  964 , height =  903
all corner 1 len =  426
all corner 2 len =  592
matched pairs num =  392
iterations =  4000
Number of pairs after filtering =  112
inbuilt =  [[ 1.42109674e-01 -7.21047181e-01  4.57689441e+02]
 [ 4.80491679e-02 -2.57451644e-01  1.64379120e+02]
 [ 3.07844309e-04 -1.57427599e-03  1.00000000e+00]]
Computed =  [[ 6.66450246e-01 -7.69769816e-02  3.19239967e+02]
 [-2.46396681e-01  7.83620457e-01  7.91178940e+01]
 [-5.69740419e-04 -1.55506792e-04  1.00000000e+00]]
shapes
(885, 789, 3)
(964, 903, 3)
transformed points =  [[[ 319.23996   79.1179 ]]

 [[ 291.1899   895.92194]]

 [[1881.8993  1400.5406 ]]

 [[1535.1642  -209.43571]]]
min, max
0 -209
1881 1400


In [32]:
ICC = [IC1, IC2]
IC = joinImages(ICC)

processing image  1
detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection method.
goodFeaturesToTrack is already using ALMS.
width =  1626 , height =  1651
width =  1605 , height =  1872
all corner 1 len =  1000
all corner 2 len =  788
matched pairs num =  980
iterations =  4000
Number of pairs after filtering =  188
inbuilt =  [[ 1.93877155e+01  3.46626143e+00 -9.11852237e+03]
 [-1.57628910e+01 -2.65508304e+00  7.26803996e+03]
 [-2.43668379e-03 -2.71073388e-04  1.00000000e+00]]
Computed =  [[ 2.88200909e+00  3.95743743e-02 -3.61977303e+03]
 [ 7.62914634e-01  2.65818080e+00 -5.87322554e+02]
 [ 1.14008215e-03  5.85405181e-05  1.00000000e+00]]
shapes
(1626, 1651, 3)
(1605, 1872, 3)
transformed points =  [[[-3619.773    -587.3226 ]]

 [[-3246.4094   3410.2668 ]]

 [[  403.9587   1677.4188 ]]

 [[  394.974     233.23567]]]
min, max
-3619 -587
1872 3410


In [24]:
IAC = [I1, I2]
IC = joinImages(IAC)

processing image  1
detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection method.
goodFeaturesToTrack is already using ALMS.
width =  1948 , height =  2136
width =  1375 , height =  1345
all corner 1 len =  1000
all corner 2 len =  731
matched pairs num =  984
iterations =  4000
Number of pairs after filtering =  170
inbuilt =  [[-4.90308820e-01 -2.99605418e-01  8.66904404e+02]
 [-1.22731513e+00 -7.43721233e-01  2.16654045e+03]
 [-5.65312082e-04 -3.48434455e-04  1.00000000e+00]]
Computed =  [[ 8.93950561e+00  4.38338678e-01 -1.56259688e+04]
 [ 2.03247349e+00  8.96887163e+00 -4.08980610e+03]
 [ 3.61705814e-03  7.39540669e-04  1.00000000e+00]]
shapes
(1948, 2136, 3)
(1375, 1345, 3)
transformed points =  [[[-15625.969     -4089.8062  ]]

 [[ -6052.5825     5482.8394  ]]

 [[   425.18372    1743.2389  ]]

 [[   397.52472      28.828356]]]
min, max
-15625 -4089
1345 5482


In [24]:
img_array_comb1 = [IC, I3]
ICf = joinImages(img_array_comb1)

NameError: name 'IC' is not defined

In [41]:
for i in range(1, 3):

    print("processing image ", i)
    image1 = images[i] 

    image_pair = [image0, image1]
      
    detected_corners, cmaps, corner_images = detectCorners(image_pair, choice)
    displayImages(corner_images, "corners")
    """
    Perform ANMS: Adaptive Non-Maximal Suppression
    Save ANMS output as anms.png
    """
    if (choice == 1):
        print("Applying ALMS.")
        detected_corners, anms_image = AdaptiveNonMaximalSuppression(image_pair, cmaps, 500)
        displayImages(anms_image, "anms_output")
    else:
        print("goodFeaturesToTrack is already using ALMS.") #review

    detected_corners0 = detected_corners[0]
    detected_corners1 = detected_corners[1]
            
    matched_pairs = getPairs(image0, image1, detected_corners0, detected_corners1, patch_size = 40, alpha = 0.9 )
    showMatches(image0, image1, matched_pairs, partition_width = 20)
    """
    Refine: RANSAC, Estimate Homography
    """
    H,filtered_matched_pairs = filterOutliers(matched_pairs, 20, 0.9, 5)
    showMatches(image0, image1, filtered_matched_pairs, partition_width = 20)
    """
    Image Warping + Blending
    Save Panorama output as mypano.png
    """
    stitched_image = stitchImagePairs(image0, image1, H)
    stitched_image = cropImage(stitched_image)
    cv2.imshow("pano.jpg", stitched_image)
    cv2.waitKey() 
    cv2.destroyAllWindows()
    cv2.imwrite("Results/pano.png", stitched_image)
    image0 = stitched_image
        

processing image  1
detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection method.
goodFeaturesToTrack is already using ALMS.
width =  807 , height =  605
width =  807 , height =  605
all corner 1 len =  429
all corner 2 len =  561
matched pairs num =  386
iterations =  4000
Number of pairs after filtering =  49
inbuilt =  [[-4.12467020e-01 -3.25468518e-01  3.34338168e+02]
 [-5.74378184e-01 -4.53123039e-01  4.65206922e+02]
 [-1.23549400e-03 -9.72509615e-04  1.00000000e+00]]
Computed =  [[ 7.51125786e-01  8.83055831e-02  2.35769935e+02]
 [-1.43518048e-01  1.00013120e+00 -1.11578408e+01]
 [-4.83287111e-04  8.51947064e-05  1.00000000e+00]]
shapes
(807, 605, 3)
(807, 605, 3)
transformed points =  [[[ 235.76993   -11.157841]]

 [[ 287.28134   744.74524 ]]

 [[ 980.8082    913.38617 ]]

 [[ 975.39575  -138.4747  ]]]
min, max
0 -138
980 913
processing image  2
detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner

In [26]:
cv2.imshow("pano.jpg", IC1)
cv2.waitKey() 
cv2.destroyAllWindows()

In [47]:
image0.shape

(1187, 1149, 3)

In [46]:
images[3].shape

(807, 605, 3)

In [53]:
gray = cv2.cvtColor(image0,cv2.COLOR_BGR2GRAY)

_,thresh = cv2.threshold(gray,5,255,cv2.THRESH_BINARY)
kernel = np.ones((5,5), np.uint8)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

cv2.imshow("pano.jpg", thresh)
cv2.waitKey() 
cv2.destroyAllWindows()

cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
# Find bounding box and extract ROI
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    print(x,y,w,h)
    ROI = image0[y:y+h, x:x+w]
    break

cv2.imshow('ROI',ROI)
cv2.waitKey()
cv2.destroyAllWindows()

0 0 1149 1187


In [49]:
image = image0
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)
cv2.imshow('ROI',thresh)
cv2.waitKey()
cv2.destroyAllWindows()

# Find contour and sort by contour area
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

# Find bounding box and extract ROI
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    ROI = image[x:x+w, y:y+h]
    break

cv2.imshow('ROI',ROI)
cv2.waitKey()
cv2.destroyAllWindows()

In [54]:
cnts

[array([[[101,   0]],
 
        [[101,  11]],
 
        [[100,  12]],
 
        ...,
 
        [[123,   3]],
 
        [[122,   2]],
 
        [[122,   0]]], dtype=int32)]

In [56]:
c = cnts[0]


In [64]:
c.reshape(-1,2)

array([[101,   0],
       [101,  11],
       [100,  12],
       ...,
       [123,   3],
       [122,   2],
       [122,   0]], dtype=int32)

In [31]:
    image1 = images[2] 

    image_pair = [image1, image0]
      
    detected_corners, cmaps, corner_images = detectCorners(image_pair, choice)
    displayImages(corner_images, "corners")

detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection method.


In [32]:


    detected_corners1 = detected_corners[0]
    detected_corners0 = detected_corners[1]
            
    matched_pairs = getPairs(image1, image0, detected_corners1, detected_corners0, patch_size = 40, alpha = 0.9 )
    showMatches(image1, image0, matched_pairs, partition_width = 20)

width =  807 , height =  605
width =  1196 , height =  1039
all corner 1 len =  429
all corner 2 len =  787
matched pairs num =  386


In [33]:
    H,filtered_matched_pairs = filterOutliers(matched_pairs, 20, 0.9, 5)
    showMatches(image1, image0, filtered_matched_pairs, partition_width = 20)


iterations =  4000
Number of pairs after filtering =  48
inbuilt =  [[-1.34594960e+00 -6.71089791e-01  8.97263198e+02]
 [-2.56094495e-01 -9.16569977e-02  1.54123872e+02]
 [-1.50887966e-03 -7.39465712e-04  1.00000000e+00]]
Computed =  [[ 2.73530660e-01 -2.81288938e-02  6.79954177e+02]
 [-3.25652761e-01  7.62050446e-01  2.09391426e+02]
 [-6.70557712e-04 -1.09794557e-04  1.00000000e+00]]


In [34]:
    """
    Image Warping + Blending
    Save Panorama output as mypano.png
    """
    stitched_image = stitchImagePairs(image1, image0, H)
    stitched_image = cropImage(stitched_image)
    cv2.imshow("pano.jpg", stitched_image)
    cv2.waitKey() 
    cv2.destroyAllWindows()
    cv2.imwrite("Results/pano.png", stitched_image)
    image0 = stitched_image

shapes
(807, 605, 3)
(1196, 1039, 3)
transformed points =  [[[ 679.95416   209.39143 ]]

 [[ 721.1512    904.50946 ]]

 [[1626.9064   1240.5297  ]]

 [[1422.5514     20.816496]]]
min, max
0 0
1626 1240
